建立連結

In [ ]:
from ldap3 import Connection, ObjectDef

conn = Connection('127.0.0.1:389', user="cn=admin,dc=bluetech,dc=corp", password="admin", auto_bind=True)
repr(conn)


"Connection(server=Server(host='127.0.0.1', port=389, use_ssl=False, allowed_referral_hosts=[('*', True)], get_info='SCHEMA', mode='IP_V6_PREFERRED'), user='cn=admin,dc=bluetech,dc=corp', password='admin', auto_bind='NO_TLS', version=3, authentication='SIMPLE', client_strategy='SYNC', auto_referrals=True, check_names=True, read_only=False, lazy=False, raise_exceptions=False, fast_decoder=True, auto_range=True, return_empty_attributes=True, auto_encode=True, auto_escape=True, use_referral_cache=False)"

In [3]:
conn.search(
    search_base='dc=bluetech,dc=corp',
    search_filter='(objectclass=person)'
)

conn.entries

[DN: cn=doraemon+givenName=A夢+sn=哆啦+uid=doraemon,ou=DST,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T11:13:48.201412,
 DN: cn=nobi nobita+givenName=大雄+sn=野比+uid=nobinobita,ou=DST,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T11:13:48.201546,
 DN: cn=minamoto shizuka+givenName=靜香+sn=源+uid=minamotoshizuka,ou=TWM,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T11:13:48.201649,
 DN: cn=honekawa suneo+givenName=小夫+sn=骨川+uid=honekawasuneo,ou=FET,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T11:13:48.201740,
 DN: cn=goda takeshi+displayName=胖虎+givenName=武+sn=剛田+uid=godatakeshi,ou=COE,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T11:13:48.201827]

查詢特定屬性

In [36]:
conn.search(
    search_base='dc=bluetech,dc=corp',
    search_filter='(objectclass=person)',
    attributes=['cn', 'sn', 'givenName','uid', 'displayName']
)
# print(conn.entries)

for entry in conn.entries:
    print(f'{entry.uid}, {entry.sn}{entry.givenName} {entry.displayName if entry.displayName else ""}')
    

doraemon, 哆啦A夢 
nobinobita, 野比大雄 
minamotoshizuka, 源靜香 
honekawasuneo, 骨川小夫 
godatakeshi, 剛田武 胖虎


查詢 ou 類型

In [49]:
conn.search(
    search_base='dc=bluetech,dc=corp',
    search_filter='(ou=*)',
)
conn.entries

[DN: ou=DST,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T12:35:31.763845,
 DN: ou=TWM,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T12:35:31.763958,
 DN: ou=FET,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T12:35:31.764043,
 DN: ou=COE,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T12:35:31.764125]

查詢 DST 內的entries

In [48]:
conn.search(
    search_base='ou=DST,o=bluetech,cn=admin,dc=bluetech,dc=corp',
    search_filter='(objectclass=*)',
)
conn.entries

[DN: ou=DST,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T12:04:25.928235,
 DN: cn=doraemon+givenName=A夢+sn=哆啦+uid=doraemon,ou=DST,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T12:04:25.928335,
 DN: cn=nobi nobita+givenName=大雄+sn=野比+uid=nobinobita,ou=DST,o=bluetech,cn=admin,dc=bluetech,dc=corp - STATUS: Read - READ TIME: 2018-05-28T12:04:25.928432]

這樣是不行的！

原本的邏輯是：「查詢base底下，有ou=dst的所有物件」，但是ldap的邏輯不是這樣

In [71]:
conn.search(
    search_base='o=bluetech,cn=admin,dc=bluetech,dc=corp',
    search_filter='&(objectclass=*)(ou=DST)',
)
conn.entries

LDAPInvalidFilterError: invalid filter

In [4]:
import sys
from ldap3 import Server, Connection, ALL, NTLM, ALL_ATTRIBUTES, ALL_OPERATIONAL_ATTRIBUTES, AUTO_BIND_NO_TLS, SUBTREE
from ldap3.core.exceptions import LDAPCursorError

server_name = '127.0.0.1:389'
domain_name = 'dc=bluetech,dc=corp'
user_name = 'cn=admin,dc=bluetech,dc=corp'
password = 'admin'

format_string = '{:40}   {}'
print(format_string.format('Group', 'Description'))

server = Server(server_name, get_info=ALL)
conn = Connection(server,
                  # user='{}\\{}'.format(domain_name, user_name),
                  user=user_name,
                  password=password,
                  # authentication=NTLM,
                  auto_bind=True)
conn.search('dc={},dc=local'.format(domain_name), '(objectclass=group)',
            attributes=[ALL_ATTRIBUTES, ALL_OPERATIONAL_ATTRIBUTES])

for e in sorted(conn.entries):
    try:
        desc = e.description
    except LDAPCursorError:
        desc = ""
    print(format_string.format(str(e.name), desc))

Group                                      Description


LDAPObjectClassError: invalid class in objectClass attribute: group

# 使用laurelin

In [19]:
from laurelin.ldap import LDAP

with LDAP('ldap://127.0.0.1:389') as ldap:
    print(ldap.base)
    results = ldap.search(base_dn=ldap.base)

LDAPObject(dn='dc=bluetech,dc=corp', attrs={})


{}